# Importing Libraries

In [1]:
import pandas as pd

# Loading Data

In [2]:
client_df = pd.read_csv('/content/drive/MyDrive/BCG DATASET/client_data.csv')
price_df = pd.read_csv('/content/drive/MyDrive/BCG DATASET/price_data.csv')

# Converting date columns into appropriate data type.

In [3]:
client_df["date_activ"] = pd.to_datetime(client_df["date_activ"], format='%Y-%m-%d')
client_df["date_end"] = pd.to_datetime(client_df["date_end"], format='%Y-%m-%d')
client_df["date_modif_prod"] = pd.to_datetime(client_df["date_modif_prod"], format='%Y-%m-%d')
client_df["date_renewal"] = pd.to_datetime(client_df["date_renewal"], format='%Y-%m-%d')

In [4]:
client_df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,f,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0


In [5]:
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


# Feature Engineering

Below is the feature engineering already performed by colleague.

In [6]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


# The feature Engineering I gonna perform.

We have listed the names of the columns which had the highest correlation with the churn column. And we will proceed using this columns to make our seconf feature.

In [7]:
forecast_col = ['id','forecast_discount_energy','forecast_cons_12m','forecast_meter_rent_12m','forecast_price_energy_peak','forecast_cons_year','margin_net_pow_ele','margin_gross_pow_ele','net_margin','pow_max','churn']

In [8]:
forcast_feat = client_df[forecast_col].copy()
forcast_feat.head()

,id,forecast_discount_energy,forecast_cons_12m,forecast_meter_rent_12m,forecast_price_energy_peak,forecast_cons_year,margin_net_pow_ele,margin_gross_pow_ele,net_margin,pow_max,churn
0,24011ae4ebbe3035111d65fa7c15bc57,0.0,0.00,1.78,0.098142,0,25.44,25.44,678.99,43.648,1
1,d29c2c54acc38ff3c0614d0a653813dd,0.0,189.95,16.27,0.000000,0,16.38,16.38,18.89,13.800,0
2,764c75f661154dac3a6c254cd082ea7d,0.0,47.96,38.72,0.087899,0,28.60,28.60,6.60,13.856,0
3,bba03439a292a1e166f80264c16191cb,0.0,240.04,19.83,0.000000,0,30.22,30.22,25.46,13.200,0
4,149d57cf92fc41cf94415803a877cb4b,0.0,445.75,131.73,0.100015,526,44.91,44.91,47.98,19.800,0


In [9]:
forcast_feat.shape

(14606, 11)

In [10]:
diff.shape

(16096, 3)

# Merging both of our Features.

In [11]:
new_fea = pd.merge(diff,forcast_feat,on='id').reset_index()
new_fea.head()

,index,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,forecast_discount_energy,forecast_cons_12m,forecast_meter_rent_12m,forecast_price_energy_peak,forecast_cons_year,margin_net_pow_ele,margin_gross_pow_ele,net_margin,pow_max,churn
0,0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916,0.0,729.06,138.95,0.100015,425,43.08,43.08,81.42,17.250,0
1,1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779,0.0,597.77,6.84,0.000000,0,24.42,24.42,61.58,13.200,0
2,2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000,30.0,1311.16,18.37,0.000000,1062,38.58,38.58,81.61,13.856,0
3,3,00114d74e963e47177db89bc70108537,-0.003994,-0.000001,0.0,1671.41,18.27,0.000000,0,29.76,29.76,157.99,13.200,0
4,4,0013f326a839a2f6ad87a1859952d227,-0.006171,0.000000,0.0,3077.34,144.86,0.101750,1760,30.00,30.00,341.58,20.000,0


In [12]:
new_fea.shape

(14606, 14)

# Splitting our data into Train and Test.

In [13]:
X = new_fea.drop(columns=['id','churn'],axis=1)
y = new_fea['churn']


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing our data for better model training.

In [15]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()


X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

# THIS PROBLEM IS BEST DEFINED AS A CLASSIFICATION PROBLEM, as we have Churn target fearures. And the motivation of this problem is to determine whether the customer will churn and how the discount will affect the customer churning.

In [16]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train_scaled, y_train)

y_pred = rf_classifier.predict(X_test_scaled)

# Model Performance Metrics

In [17]:
from sklearn.metrics import accuracy_score,classification_report

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8997262149212868


I believe having provided the data, the accuracy is satisfactory and will work as there wasm't a strong pattern or relationship between the features and churn result.

In [20]:
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      2630
           1       0.44      0.01      0.03       292

    accuracy                           0.90      2922
   macro avg       0.67      0.51      0.49      2922
weighted avg       0.86      0.90      0.86      2922



# Classification report is the best measure to evaluate the performance of this model. It gives a complete classification report of both the labels.

# Advantages of RandomForest Classifier in this case:

* It make use of esemble learning which is a technique had trains multiple decision trees of subsets our data with replacement.
* We can a average result from all the decision trees used in the RandomForest Classifier. Thus Improves the performance and the ability of the model to adjust with the given data.


# Disadvantage :

* The Training time is slightly more, but not too large.